In [1]:
import numpy as np
import pandas as pd

import requests, json

In [2]:
API_REQUEST_URL = 'https://climatology.weathersourceapis.com/v2'

# An hour of weather climatology for a DMA
API_CLIMATOLOGY_DMAS_HOURLY = '/dmas/{dmaId}/hours/{timestamp}/?fields={fields}&unitScale=IMPERIAL'


REQUEST_TEMPLATE = {      
    "dmaId":      "819",  # DMA Id of Seattle - Tacoma area
    "timestamp":  "",     # the three digit county FIPS # as a string
    "fields":     "popular"
}

API_KEY = "p9zGb5GyMFaGRvaBR4lWyADlV"

In [3]:
headers = {
    'accept': 'application/json',
    'X-API-KEY': f'{API_KEY}',
}

def request_hourly_info(timestamp = None,
                        endpoint_url = API_REQUEST_URL, 
                        endpoint_action = API_CLIMATOLOGY_DMAS_HOURLY,
                        request_template = REQUEST_TEMPLATE,
                        headers = headers):
    
    request_template['timestamp'] = timestamp

    # compose the request
    # print(endpoint_action.format(**request_template))
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [4]:
from tqdm import tqdm

In [5]:
from datetime import timedelta, datetime, timezone

# datetime.fromisoformat('2011-11-04T00:05:23') + timedelta(hours=2)

hours = 24 * 365 * 3

rs = {"data":[]}

for i in tqdm(range(hours)):
    ts = (datetime(2010, 1, 1, 0, 0, tzinfo=timezone(timedelta(hours=8))) + timedelta(hours=i)).isoformat()
    # ts = '2010-12-20T19:00:00+08:00'
    response = request_hourly_info(timestamp=ts)
    response["climatology"][0]['ts'] = ts
    rs['data'].append(response["climatology"][0])
with open("cli_2010.json", "w") as outfile:
    outfile.write(json.dumps(rs,indent=4))

100%|██████████| 26280/26280 [1:29:30<00:00,  4.89it/s]  


In [6]:
df = pd.read_json('cli_2010.json')
df = pd.json_normalize(df['data'])
df

,cldCvr,cldCvrSd,feelsLike,feelsLikeSd,precip,precipSd,relHum,relHumSd,snowfall,snowfallSd,temp,tempSd,timestamp,windDir,windSpd,windSpdSd,ts
0,77.0,32.19,34.29,6.71,0.05,0.04,90.9,8.0,0.0,0.02,37.46,6.37,2009-12-31T16:00:00Z,165.54,5.22,4.01,2010-01-01T00:00:00+08:00
1,76.6,31.79,34.49,6.47,0.06,0.06,90.8,7.7,0.0,0.02,37.77,6.22,2009-12-31T17:00:00Z,166.10,5.49,4.21,2010-01-01T01:00:00+08:00
2,75.8,31.99,35.65,5.92,0.05,0.05,89.9,8.0,0.0,0.01,38.91,5.61,2009-12-31T18:00:00Z,169.18,5.71,4.37,2010-01-01T02:00:00+08:00
3,76.9,32.57,37.07,5.68,0.04,0.03,88.1,9.7,0.0,0.01,40.28,5.18,2009-12-31T19:00:00Z,172.36,5.87,4.49,2010-01-01T03:00:00+08:00
4,77.2,31.94,38.19,5.49,0.05,0.03,86.0,10.4,0.0,0.01,41.36,4.95,2009-12-31T20:00:00Z,178.39,6.10,4.78,2010-01-01T04:00:00+08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26275,74.8,31.97,34.70,6.34,0.04,0.04,90.9,8.1,0.0,0.02,37.91,6.02,2012-12-30T11:00:00Z,166.57,5.37,4.19,2012-12-30T19:00:00+08:00
26276,75.0,31.47,34.60,6.38,0.04,0.04,90.8,8.1,0.0,0.03,37.75,6.09,2012-12-30T12:00:00Z,166.30,5.29,4.18,2012-12-30T20:00:00+08:00
26277,77.4,32.20,34.50,6.43,0.04,0.03,91.0,7.8,0.0,0.02,37.67,6.18,2012-12-30T13:00:00Z,164.61,5.26,4.03,2012-12-30T21:00:00+08:00
26278,76.8,32.30,34.42,6.47,0.05,0.04,91.0,7.8,0.0,0.02,37.56,6.27,2012-12-30T14:00:00Z,163.04,5.24,4.02,2012-12-30T22:00:00+08:00
